In [ ]:
import boto3

In [ ]:
s3 = boto3.resource('s3')

In [ ]:
## if on prem- you need to manualy set the machines time to gmt+0
## -2 hours
# thanks ITO
# 
# date -s '07 June 2018 13:24:00'

In [ ]:
for bucket in s3.buckets.all():
    print(bucket.name)

In [ ]:
ec2 = boto3.client('ec2')

In [ ]:
IAM_profile = {
        'Arn': 'arn:aws:iam::680041685499:instance-profile/SSM-access'
    }

In [ ]:
tags = [
                {
                    'ResourceType':'instance',
                    'Tags': [
                        {
                            'Key': 'use',
                            'Value': 'boto3'
                        },
                    ]
                }]

* if you want to run an isntance with an IAM role - the boto credentials need to have some admin rights
https://stackoverflow.com/questions/27266400/launching-ec-instance-a-client-error-unauthorizedoperation-occurred-when-ca

For a more in depth explination:
* https://aws.amazon.com/blogs/security/granting-permission-to-launch-ec2-instances-with-iam-roles-passrole-permission/

In [ ]:
isntance = ec2.run_instances(
        ImageId='ami-ca0135b3',
        MinCount=1,
        MaxCount=1,
        KeyName='testing-jeroen',
        InstanceType='t1.micro',
        SubnetId='subnet-36539650',
        TagSpecifications=tags,
        IamInstanceProfile=IAM_profile
        )

In [ ]:
isntance

In [ ]:
OutputLocation={
        'S3Location': {

        }
}

In [ ]:
def execute_commands_on_linux_instances(client, commands, instance_ids):
    """Runs commands on remote linux instances
    :param client: a boto/boto3 ssm client
    :param commands: a list of strings, each one a command to execute on the instances
    :param instance_ids: a list of instance_id strings, of the instances on which to execute the command
    :return: the response from the send_command function (check the boto3 docs for ssm client.send_command() )
    """

    resp = client.send_command(
        DocumentName="AWS-RunShellScript", # One of AWS' preconfigured documents
        Parameters={'commands': commands},
        InstanceIds=instance_ids,
        OutputS3BucketName='blt-emr-testing',
        OutputS3KeyPrefix='boto-logs'
    )
    return resp

In [ ]:
# Example use:
ssm_client = boto3.client('ssm') # Need your credentials here
commands = ['echo "hello world"']
instance_ids = ['i-009dd75468e14dd3e']
execute_commands_on_linux_instances(ssm_client, commands, instance_ids)

In [ ]:
! aws ssm describe-instance-information --output text

In [ ]:
! aws ssm describe-instance-information